<center><h1>QBUS6860 - Visual Data Analytics</h1></center>

# Tutorial 09 - Visualizing networks

## Learning objectives and tasks:

1. Plotting basic network graphs with <code>networkx</code>
2. Interpreting characteristics of a network
3. Chord Diagram using <code>bokeh</code> and <code>holoviews</code>
4. Sankey Diagram using <code>bokeh</code> and <code>holoviews</code>

## Task 1: Drawing a basic network

Solve the following problem using network visualisation in Python:

### Step 1.  Understand the data

A company, producing umbrellas has headquarters in China, keeps warehouses in China, UK, Ireland, USA, Canada, India and Brazil. In each country, it stores the following number of umbrellas (in millions of units):

* China=150
* UK=25
* USA=75
* India=15
* Canada=40
* Ireland=30
* Brazil=5

The supply chain of the company is organized as follows:

* From China, the umbrellas go to the UK, the USA, India and Brazil directly
* Umbrellas get to Ireland from the UK
* Umbrellas get to Canada from the USA

Using network visualisation, plot the company supply chain using 

(1) continuous color to capture the stock volume

(2) [if time permits] discrete color to capture the supply chain groups 


First you need need install the following packages:

In [ ]:
#pip install decorator==5.0.9

### Step 2. Start defining the network

We have two key elements of network (edges and nodes). Next, we need to build our graph (instance in memory).<br>

<code>**networkx.from_pandas_edgelist(df, source='source', target='target', edge_attr=None, create_using=None, edge_key=None)**</code>.<br>

Parameters:
 - **df** (*Pandas DataFrame*): An edge list representation of a graph

 - **source** (*str or int*): A valid column name (string or integer) for the source nodes (for the directed case).

 - **target** (*str or int*): A valid column name (string or integer) for the target nodes (for the directed case).
<!--  - **edge_attr** (*str or int, iterable, True, or None): A valid column name (str or int*) or iterable of column names that are used to retrieve items and add them to the graph as edge attributes. If True, all of the remaining columns will be added. If None, no edge attributes are added to the graph. -->
 - **create_using** (*NetworkX graph constructor, optional (default=nx.Graph)*): Graph type to create. If graph instance, then cleared before populated.

<!--  - **edge_key** (*str or None, optional (default=None)*): A valid column name for the edge keys (for a MultiGraph). The values in this column are used for the edge keys when adding edges if create_using is a multigraph. -->

- **edge_attr** :adding weight for each edge

Build the graph object by <code>nx.from_pandas_edgelist</code>

Plot the supply chain network providing a continuous color scale with cmap (we will use Blue color, if you want to use Red colour write "Reds" intead of "Blues", if you want Green color, use "Greens" etc.):

<code>**draw(G, pos=None, ax=None, **kwds)**</code><br>
Parameters:
 - **G (*graph*)**: A networkx graph

 - **pos (*dictionary, optional*)**: A dictionary with nodes as keys and positions as values. If not specified a spring layout positioning will be computed. See networkx.drawing.layout for functions that compute node positions.

 - **ax (*Matplotlib Axes object, optional*)**: Draw the graph in specified Matplotlib axes.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

 
# Build a dataframe with your connections
# Remember that the "from" part should be linked with "to" part
# In our case, China is linked to India; UK to China; USA to Canada; China to USA; UK to Ireland; and Brazil to China

# Yuning's comment
# You can think this DataFrame contain the graph topology information
# Connection or Edges 
# create nodes for the graph
# from-to: a relationship between the nodes
# each row: an edge in the graph


df = pd.DataFrame({ 'from':['China', 'UK', 'USA','China','UK', 'Brazil'], 
                   'to':['India', 'China', 'Canada','USA','Ireland','China']})
 
# Add data frame with characteristics for your nodes (in our case, stock volumes)

# Yuning:
# This carac contain all node features informations.
# Features or Node
# name of nodes ---> the number of stock in each warehouse 
carac = pd.DataFrame({ 'ID':['China', 'UK', 'USA','India','Canada','Ireland','Brazil'], 
                      'myvalue':['150','25','75','15','40','30','5'] })
 
# Build your graph
# Yuning: fisrtly create an empty graph using nx.Graph(), then add edge with the data in df to the empty graph
G=nx.from_pandas_edgelist(df, 'from', 'to', create_using=nx.Graph() )
 
# The order of the node for networkX is the following order:
G.nodes()
# The outcome for this step: NodeView(('China', 'India', 'UK', 'USA', 'Canada', 'Ireland', 'Brazil'))

# We cannot give the 'myvalue' column to netowrkX directly: we need to arrange the order.
 
# We need to reorder carac to assign the correct color to each node (the larger the stock, the deeper the colour)
carac= carac.set_index('ID') # Setting the row index as ID column
carac=carac.reindex(G.nodes()) # Reset index order by Node view
 
# Plot the supply chain network providing a continuous color scale with cmap 
#(we will use Blue color, if you want to use Red colour write "Reds" intead of "Blues"
# if you want Green color, use "Greens", etc.):
nx.draw(G, with_labels=True, node_color=carac['myvalue'].astype(int), cmap=plt.cm.Blues)
plt.savefig('Network1.png')
plt.show()

Yuning's comment: The efficiency of this visualization is not good actually, coz the colors are not distinguishable, e.g. comparing UK, Canada and Ireland.

Be careful to use it when you have more nodes.

### Step 3. Using colors for groups

Now let's plot the same thing using groupings and discrete color

* USA and Canada form a group
* UK and Ireland form a group
* China, India, and Brazil form their own groups

Each group should be plotted using a different colour


we need to transform our categorical column in a numerical value: group1->0, group2->1...<br>
<code>**pandas.Categorical(values, ...)**</code>
Parameters:
- **values (*list-like*)**: The values of the categorical. If categories are given, values not in categories will be replaced with NaN.

<code>**Series.cat.categories**</code>:<br>
The categories of this categorical. Setting assigns new values to each category (effectively a rename of each individual category). The assigned value has to be a list-like object. All items must be unique and the number of items in the new categories must be the same as the number of items in the old categories.<br>
<br>
<code>**Series.cat.codes**</code>:<br>
Return Series of codes as well as the index and get the numerical values.

In [ ]:
# libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
 
# Build a dataframe with your connections (you can copy-paste from the previous graph code)
df = pd.DataFrame({ 'from':['China', 'UK', 'USA','China','UK', 'Brazil'], 
                   'to':['India', 'China', 'Canada','USA','Ireland','China']})
 
# Add data frame with characteristics for your nodes (in our case, supply chain groups)
carac = pd.DataFrame({ 'ID':['China', 'UK', 'USA','India','Canada','Ireland','Brazil'], 
                      'myvalue':['group1','group2','group3','group4','group3', 'group2', 'group5'] })
 
# Build your graph
G=nx.from_pandas_edgelist(df, 'from', 'to', create_using=nx.Graph())
 
# The order of the node for networkX is the following order:
G.nodes()
# We cannot give the 'myvalue' column to netowrkX directly, we need to arrange the order.
 
# Now we reorder carac to assign the correct color to each node
carac = carac.set_index('ID')
carac = carac.reindex(G.nodes())
 
# And we need to transform our categorical column in a numerical value: group1->1, group2->2...
carac['myvalue']=pd.Categorical(carac['myvalue'])
carac['myvalue'].cat.codes
 
# Custom the nodes (we will use node size = 800, you can select a lower or higher number):
nx.draw(G, with_labels=True, node_color=carac['myvalue'].cat.codes, cmap=plt.cm.Set1, node_size=800)
plt.savefig('Network2.png')

## Task 2: Interpreting characteristics of a network

Centrality algorithms measure how important a node is. This is not a clear definition, but it’s useful when we want to identify important (highly connected, in some cases) nodes or potential bottlenecks (barely connected, in some cases) in network.

A walk is a path which can go through a node several times. Centrality measures vary with the type of walk considered and the way of counting them.

<code>networkx</code> provides a series of functions https://networkx.org/documentation/stable/reference/functions.html and algorithms https://networkx.org/documentation/stable/reference/algorithms/index.htmlto understand and analyze a network.

In [ ]:
# Expanding on the previous network from Task 1 to include more nodes and countries
df2 = pd.DataFrame({'from':['Brazil', 'Argentina', 'Brazil', 'Chile', 'Canada', 'Argentina', 'Canada', 'Russia', 'Mexico', 'India'], 
                   'to':['USA', 'Chile', 'Chile', 'Peru', 'Mexico', 'Bolivia', 'Argentina', 'China', 'USA', 'Australia']})
edf = pd.concat([df, df2])
G = nx.from_pandas_edgelist(edf, 'from', 'to', create_using=nx.Graph())
nx.draw(G, with_labels=True, node_size=1000, node_color='grey')

### Step 1: Summary statistics

Methodology adapted from: https://towardsdatascience.com/introduction-to-graphs-part-1-2de6cda8c5a5

In [ ]:
# Compute number of nodes, edges and metrics
nodes = G.number_of_nodes()
edges = G.number_of_edges()
degrees = list(G.degree())

avg_degree = np.mean(np.array(degrees)[:,1].astype(float))
med_degree = np.median(np.array(degrees)[:,1].astype(float))
max_degree = max(np.array(degrees)[:,1].astype(float))
min_degree = np.min(np.array(degrees)[:,1].astype(float))

print("Number of nodes: " + str(nodes))
print("Number of edges: " + str(edges))
print("Maximum degree of a node: " + str(max_degree))
print("Minimum degree of a node: " + str(min_degree))
print("Average degree of nodes: " + str(avg_degree))
print("Median degree of nodes: " + str(med_degree))

> Ask yourself: What is your interpretation based on the summary statistics above?

### Step 2.1. Degree centrality
Degree centrality counts the number of walks of length 1 ending at node i. It is used to identify the most influential node. All the values are standardarised score.

In [ ]:
degree_centrality = nx.degree_centrality(G)
dict(sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True))

In [ ]:
c_degree = list(degree_centrality.values())
nx.draw(G, cmap = plt.get_cmap('tab20'), node_color = c_degree, node_size=1000, with_labels=True)
plt.title('Degree Centrality')

### Step 2.2. Eigenvector centrality
Eigenvector Centrality is the number of walks of infinite length ending at node i. This gives more importance to nodes with well-connected neighbors.

In [ ]:
eigenvector_centrality = nx.eigenvector_centrality(G)
dict(sorted(eigenvector_centrality.items(), key=lambda item: item[1], reverse=True))

In [ ]:
c_eigenvector = list(eigenvector_centrality.values())
nx.draw(G, cmap = plt.get_cmap('tab20'), node_color = c_eigenvector, node_size=1000, with_labels=True)
plt.title('Eigenvector Centrality')

### Step 2.3. Closeness centrality
Closeness Centrality detects nodes that are can spread information efficiently through a graph. It is inversely proportional to the sum of lengths of the shortest paths to other nodes.

In [ ]:
closeness_centrality = nx.closeness_centrality(G)
dict(sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True))

In [ ]:
c_closeness = list(closeness_centrality.values())
nx.draw(G, cmap = plt.get_cmap('tab20'), node_color = c_closeness, node_size=1000, with_labels=True)
plt.title('Closeness Centrality')

### Step 2.4. Betweenness centrality
Betweenness Centrality detects the amount of influence a node has over the flow of information in a graph. It is often used to find nodes that serve as a bridge from one part of a graph to another

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G)
dict(sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True))

In [ ]:
c_betweenness = list(betweenness_centrality.values())
nx.draw(G, cmap = plt.get_cmap('tab20'), node_color = c_betweenness, node_size=1000, with_labels=True)
plt.title('Betweenness Centrality')

### Centrality in summary:

In [ ]:
# Set node colors
cmap = plt.get_cmap('tab20')

# Set node sizes
node_sizes = 1000

# Set overall figure size
f = plt.figure(figsize=(20,20))
f.tight_layout()

# Subplot 1
plt.subplot(2, 2, 1)
nx.draw(G, with_labels=True, node_color=c_degree, cmap=cmap, 
        node_size = node_sizes, edgecolors='gray')
plt.title('Degree Centrality', fontsize=18)

# Subplot 2
plt.subplot(2, 2, 2)
nx.draw(G, with_labels=True, node_color=c_eigenvector, cmap=cmap, 
               node_size = node_sizes, edgecolors='gray')
plt.title('Eigenvector Centrality', fontsize=18)

# Subplot 3
plt.subplot(2, 2, 3)
nx.draw(G, with_labels=True, node_color=c_closeness, cmap=cmap, 
            node_size = node_sizes, edgecolors='gray')
plt.title('Closeness Centrality', fontsize=18)

# Subplot 4
plt.subplot(2, 2, 4)
nx.draw(G, with_labels=True, node_color=c_betweenness, cmap=cmap, 
            node_size = node_sizes, edgecolors='gray')
plt.title('Betweenness Centrality', fontsize=18)

> Ask yourself:
> How would you describe the characteristics of a country (of your choice) based on the visualizations and metrics above?

## Task 3:  Chord Diagram using <code>bokeh</code> and <code>holoviews</code>

As demonstrated in Lecture example, we can use <code>bokeh</code> along with <code>holoviews</code> to draw Chord Diagrams.
Here is a quick example

### Step 1.   Importing Packages and Preparing data

We will use <code>bokeh</code>'s own datasets.

In [ ]:
# pip install holoviews

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data

hv.extension('bokeh')
hv.output(size=200)


# data set, transfer to hv.Dataset format
nodes = hv.Dataset(pd.DataFrame(data['nodes']), 'index') 
links = pd.DataFrame(data['links'])

In [ ]:
data

In [ ]:
nodes

As been seen, the variable <code>nodes</code> is a <code>holoviews</code> defined <code>Dataset</code> constructed from the given DataFrame. The purpose of doing so is to add node labels and other information such that the Dataset contains with a key dimension of node indices.

More **details** in <code>holoviews.Dataset()</code>, Please check https://holoviews.org/getting_started/Tabular_Datasets.html

You may explore the information inside the data and link as follows

In [ ]:
# The environment doesnt have holoview package so you have to install it yourself
# pip install holoviews

As been seen, the variable <code>nodes</code> is a <code>holoviews</code> defined <code>Dataset</code> constructed from the given DataFrame. The purpose of doing so is to add node labels and other information such that the Dataset contains with a key dimension of node indices.

You may explore the information inside the data and link as follows

In [ ]:
nodes_DataFrame = pd.DataFrame(data['nodes'])
nodes_DataFrame.head()

In [ ]:
links = pd.DataFrame(data['links']) #value is the info from the edge
links.info() 

### Step 2.  Draw the diagram with added feature

In [ ]:
# chord diagram
chord = hv.Chord((links, nodes)).select(value=(5, None))
# no more graph object and the data consists link and nodes together,
# select the values of links greater or equal than 5
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('source').str(), 
               labels='name', node_color=dim('index').str()))

## Task 4:  Sankey Diagram using <code>bokeh</code> and <code>holoviews</code>

In this example (adopted from holoviews examples), we want to show Careers paths. The following data is extracted from the 2010 Royal Society policy report entitled “The Scientific Century: securing our future prosperity” https://royalsociety.org/~/media/royal_society_content/policy/publications/2010/4294970126.pdf. 

In fact, we have known that 53% of the PhD Students went to a Career Outside Science, 47% of the them got an Early Career Researcher. 17% of those had been in an Early Career Researcher had a Non-Academic Research and 30% became a Permanent Research Staff. Only 22% of the last category got a Professor position which represents only 0.45% of the PhD students.

source: https://malouche.github.io/notebooks/Sankey_graphs.html

### Step 1. Import Packages and Preparing data

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data

hv.extension('bokeh')
hv.output(size=100)

In [ ]:
careers = ["PhD", "Career Outside Science",  "Early Career Researcher", "Research Staff",
         "Permanent Research Staff",  "Professor",  "Non-Academic Research"]

nodes = pd.DataFrame({'label':careers})  # make a DataFrame for nodes

nodes = hv.Dataset(nodes, 'index')     # add an index column explicitly

According to the order, we know that "PhD" has index value 0, "Career Outside Science" has index value 1, ...  Based on these indices and the information, the career path edge can be defined as three elements tuples instead of using rows of a DataFrame.  In the following code, there is an edge from "PhD" (index 0) to "Career Outside Science" (index 1) with 53%, which is represented as a tuple (0, 1, 53).


In [ ]:
edges = [
    (0, 1, 53), 
    (0, 2, 47), 
    (2, 6, 17), 
    (2, 3, 30), 
    (3, 1, 22.5), 
    (3, 4, 3.5), 
    (3, 6, 4.), 
    (4, 5, 0.45)   
]
value_dim = hv.Dimension('Percentage', unit='%')  # explain the value as percentage

### Step 2.  Draw the diagram

Once the data is ready, it is straightforward to draw the diagram

In [ ]:
hv.Sankey((edges, nodes), ['From', 'To'], vdims=value_dim).options(
    label_index='label', label_position='left', width=900, height=600, edge_color_index='To'
)

<hr style="border:1px solid gray"> <center><h1> Alternative Task </h1></center><hr style="border:1px solid gray">

## Task:  Graph layout in <code>networkx</code>

Adopted from the example in https://towardsdatascience.com/customizing-networkx-graphs-f80b4e69bedf.

### Step 1.  Define the Graph Data  

We normally define all the edges and node information (features) in two different DataFrames.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib


# Specify data and attributes
edges = pd.DataFrame({'from': ['A', 'A', 'A', '1', '2', '3', '1', 'Center', 'Center', 
                                       'Center', 'Center', 'Center', 'Center', 'Center'], 
                              'to': ['B', 'C', 'D', 'C', 'C', 'A', '3', '1', '3', '2', 'A', 'B', 
                                     'C', 'D']})

# Create DF for node characteristics
nodes = pd.DataFrame({'ID':['A', 'B', 'C', 'D', '1', '2', '3', 'Center'], 
                      'type':['Letter','Letter', 'Letter', 'Letter', 'Number', 'Number', 
                              'Number', 'Center']})


In [ ]:
# Let's check what we have in edges and nodes
edges, nodes

### Step 2.   Creating <code>networkx</code> Graph Object

Graph can be simply created from edge information stored in a <code>pandas</code> DataFrame

In [ ]:
# Create graph object
G = nx.from_pandas_edgelist(edges, 'from', 'to', create_using=nx.Graph())

### Step 3. Preparing Node Characteristic Information

<code>pandas.Categorical(values, ...)</code>

**Parameters**:

**values**: list-like
- The values of the categorical. If categories are given, values not in categories will be replaced with NaN.

<code>pandas.Categorical.codes</code>

**property** Categorical.codes

The category codes of this categorical.

Codes are an array of integers which are the positions of the actual values in the categories array.

There is no setter, use the other categorical methods and the normal item setter to change values in the categorical.

Returns
ndarray[int]

A non-writable view of the codes array.

In [ ]:
# Make types into categories
nodes= nodes.set_index('ID')
nodes=nodes.reindex(G.nodes())

nodes['type']=pd.Categorical(nodes['type'])
nodes['type'].cat.codes #transfer the categorical info to numbers

### Step 4. Preparing Drawing Information and Outputing Graphs

In [ ]:
# Set node colors
cmap = matplotlib.colors.ListedColormap(['dodgerblue', 'lightgray', 'darkorange'])

# Set node sizes
# letter nodes are smaller than others
node_sizes = [4000 if entry != 'Letter' else 1000 for entry in nodes.type]

# Set overall figure size
f = plt.figure(figsize=(20,20))
f.tight_layout()

# Create Layouts

# Subplot 1
plt.subplot(2, 2, 1)
nx.draw(G, with_labels=True, node_color=nodes['type'].cat.codes, cmap=cmap, 
        node_size = node_sizes, edgecolors='gray')
plt.title('Spring Layout (Default)', fontsize=18)

# Subplot 2
plt.subplot(2, 2, 2)
nx.draw_random(G, with_labels=True, node_color=nodes['type'].cat.codes, cmap=cmap, 
               node_size = node_sizes, edgecolors='gray')
plt.title('Random Layout', fontsize=18)

# Subplot 3
plt.subplot(2, 2, 3)
nx.draw_shell(G, with_labels=True, node_color=nodes['type'].cat.codes, cmap=cmap, 
            node_size = node_sizes, edgecolors='gray')
plt.title('Shell Layout', fontsize=18)

# Subplot 4
plt.subplot(2, 2, 4)
nx.draw_spectral(G, with_labels=True, node_color=nodes['type'].cat.codes, cmap=cmap, 
            node_size = node_sizes, edgecolors='gray')
plt.title('Spectral Layout', fontsize=18)

**Spring layout**
https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html

Position nodes using Fruchterman-Reingold force-directed algorithm.

The algorithm simulates a force-directed representation of the network treating edges as springs holding nodes close, while treating nodes as repelling objects, sometimes called an anti-gravity force. Simulation continues until the positions are close to an equilibrium.

**Random layout**

https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.random_layout.html

Position nodes uniformly at random in the unit square.

For every node, a position is generated by choosing each of dim coordinates uniformly at random on the interval [0.0, 1.0).

NumPy (http://scipy.org) is required for this function.

**Shell layout**

https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.shell_layout.html

Position nodes in concentric circles.

**Spectral layout**

https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spectral_layout.html

Position nodes using the eigenvectors of the graph Laplacian.

Using the unnormalized Laplacian, the layout shows possible clusters of nodes which are an approximation of the ratio cut. If dim is the number of dimensions then the positions are the entries of the dim eigenvectors corresponding to the ascending eigenvalues starting from the second one.

### Summary

1. A graph can be created in <code>networkx</code> according to a DataFrame whose rows correspond to the edges in the graph and two columns define the nodes.

2. The nodes of the graph is also defined by another DataFrame whose Index should be same as the node "names" (or IDs) in the edge DataFrame columns, and a categorical column as node types.  Dont forget convert the categorical values (sometimes string values) into numeric values as code.

3. To draw the graph, the characteristic or style such as node colors, node sizes should be provided to the drawing funciton <code>networkx.draw</code> or one of four defined layouts such as <code>draw</code>, <code>draw_random</code>, <code>draw_shell</code> and <code>draw_spectral</code>. 